# SSM Project
### Author: Abhinna Sundar 

###### Purpose: This code generates a single binned lightcurve from the pv extensions of all SSM event files placed in the same folder along with stare intervals arranged in ascending order
##### Inputs: inttime: binning time in seconds
##### Outputs: .txt file containing binned lightcurve, .txt file containing all stare intervals

In [ ]:
from numpy import *
from pylab import *
from astropy.io import fits
from glob import glob
import os

filenames = sorted(glob("D:/Summer Project/AstroSat Project/Data/*.fits"))

inttime = int(input("Enter binning time: "))

outfile = open("Orbit_xxxxx_lcurve_binned_stare_" + str(inttime) + "s.txt", "w")

output_MJD = []
output_events = []
stares = open("Orbit_xxxxx_stares_"+str(inttime)+"s.txt", "w")
for filename in filenames:
	hdulist = fits.open(filename)
	Orbit_number= hdulist[0].header[8]    
	outfile = open("Orbit_xxxxx_hdulist.txt", 'w')
	hdulist.info(output=outfile)
	with open("Orbit_xxxxx_hdulist.txt") as infile:
		lines = infile.read().splitlines()
	selectlist = []
    
    ####
    #### Selecting from 'pv' extensions from a single event file, since they are the valid events
    #### A single event file has a lot of 'pv' extensions, thus, there are many data points within them
    #### We only take the minimum and maximum time durations in the stares. See below, for the algorithm 
    ####
    
	for i in lines:
		if i[11:13] == "pv":
			selectlist.append(int(i[1:3]))
	JD = []
	for hduno in selectlist:
		temp = array(hdulist[hduno].data.field("SSM_PL_EVTTIME_MJD"))
		if len(temp) > 0:
			stares.write(str(min(temp))+" "+str(max(temp))+" "+ str(int(float(Orbit_number))) +"\n") 
		for m in temp:
			JD.append(m)
            
            #####
            ##### Stares would be in format ( min(JD) , max(JD))   for different 'FITS files' in the event files folder         
            #####                           (                  )   
            #####                           (                  )
            
            ####
            #### Binning the time scale data
            ####
            
	JD = sort(JD)
	events = ones((len(JD)))
	JD_temp = (JD - JD[0])*86400
	length = int(JD_temp[len(JD_temp)-1])
	JD_binned = linspace(0,length,(length+1)//inttime)
	JD_bin = []
	events_bin = []
    
    ####
    #### Counting the number of data in a particular bin, and binning the data
    ####
    
	for i in range(len(JD_binned)-1):
		temp = 0
		count_no = 0
		for j in range(len(JD_temp)):
			if JD_binned[i] <= JD_temp[j] <= JD_binned[i+1]:
				temp = JD_binned[i]
				count_no += events[j]
			else:
				pass
		if temp == 0:
			JD_bin.append(JD_binned[i])
			events_bin.append(0)
		else:
			JD_bin.append(temp)
			events_bin.append(count_no)
		prog = (float(i)/float(len(JD_binned)-1))*100
		sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
		sys.stdout.flush()
	sys.stdout.write("\n")
	MJD = (array(JD_bin)/86400)+JD[0]
	with open("Orbit_xxxxx_lcurve_binned_stare_" + str(inttime) + "s.txt", 'a') as f:
    		for k in range(len(MJD)):
        		print(str(MJD[k])+' '+str(events_bin[k]), file=f)
                
                ####
                #### lcurve_binned_stare_()s.txt would be in the form
                ####                    ( MJD[i] , counts[i] )
                ####                    (                    )
                ####                    (                    )
                ####                                            where the MJD is the binned data, equally spaced &
                ####                                            counts are the no of data points present in the bin.
                ####                                              

stares.close()

staresref = loadtxt("Orbit_xxxxx_stares_"+str(inttime)+"s.txt", float)

#####
##### Sorting lists of (list1= max(JD), list2 = min(JD)) for different FITS files from the file stares created above
##### The sortlist() arranges the list2 in order s.t. the value corresponding to the list1 is in increasing order 

##def sort_list(list1, list2,list3): 
##    zipped_pairs = zip(list2, list1, list3) 
##    z = [x for _, x in sorted(zipped_pairs)]       
##    return z



Orbit_number=staresref[:,2]
init = staresref[:,0]
final = staresref[:,1]

#final = sort_list(final,init,Orbit_number)
I, F, O = zip(*sorted(zip(init, final, Orbit_number)))

init = list(I)
final = list(F)
Orbit_number = list(O)

####
#### Deleting the init() values, where the difference in the consequtive init values is less than the binning time(inttime)
####

i=0
while i < len(init)-1:
	if init[i+1]-init[i] < float(inttime)/86400:
		del(init[i+1])
		del(final[i+1])
		del(Orbit_number[i+1])        
	i += 1
    
####    
#### Saving the file stares_().txt as 2D array [[1,2]
####                                            [3,4]    where 1,3,5 are init and 2,4,6 are final MJD values
####                                            [5,6]]
####

stares = zeros((len(init),3))
stares[:,0] = init
stares[:,1] = final
stares[:,2] = Orbit_number

#print(stares)
savetxt("Orbit_xxxxx_stares_"+str(inttime)+"s.txt", stares)

##### Purpose: This code fits a fred signature to the combined binned lightcurve, once for each stare
##### Inputs: .txt file containing binned lightcurve, .txt file containing stare intervals. (Both are outputs of generate_lcurve_multi_stare.py), limit of redchisq above which to reject fits
##### Outputs: Plots of data and fit for each stare, .txt file containing confidence values for each fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys

def sort_list(list1, list2): 
    zipped_pairs = zip(list2, list1) 
    z = [x for _, x in sorted(zipped_pairs)]       
    return z 

def const(t, c):
    return  t*0+ c #+minvalue

def res(var, t, data,eps_data):
    c= var[0]
    model = t*0 + c #+minvalue
    return (data-model) / eps_data

def residual(variables, t, data, eps_data):
    tau1 = variables[0]
    tau2 = variables[1]
    t_s  = 0 #variables[2]
    model = minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))
    return (data-model) / eps_data

def func(t,tau1,tau2,t_s=0):
	return minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))


filename = '1_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")
sizesname = '1_stares_5s.txt'#input("Enter path to file containing stare intervals: ")
redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

count = 0

while count < len(stares):
	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[count].split()[0])
	end = float(stares[count].split()[1])
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:
			xdata.append(float(p[0]))
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)
		xdata = sorted(xdata)
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)
		var = np.median(ydata)
		sigma= np.std(ydata) 
		print('stddev = {}'.format(sigma), 'mean= {}'.format(var))         
		value= var-2*sigma
		print(count+1, '\t',value)        
		indices=np.where(ydata>value)
#		print(indices)    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
#		print(len(xfinal), len(xdata))        
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)     
#		print(len(yfinal), len(ydata))        
		out = leastsq(res, var, args=(xfinal,yfinal,100))        
		yfit = const(xfinal, out[0][0])#,out[0][1])#,out[0][2])  
#		print(out)        
        
#		variables = [0.01, 0.00015]#,0]
#		out = leastsq(residual, variables, args=(xdata, ydata, 1000))
#		yfit = func(xdata,out[0][0],out[0][1])#,out[0][2])
		y1 = yfinal
#		print('\n')        
#		print(ydata,'\n',xdata+starttime,'\n','length of data',len(ydata),',',len(xdata))        
		y2 = yfit
		ychi = []        
		for i in range(len(xfinal)):
			if y1[i] != 0:
				ychi.append((y1[i]-y2[i])**2/(y1[i]))
		chisq = sum(ychi)
		contri= (y1-y2)**2/(y1)
#		for j in range(len(xdata)):
#			if y1[j] ==0:
#				contri.append(float(0)) 
#			else:
#				contri.append(chisq/ydata)
		y0=np.zeros(shape= np.shape(contri)) 


            
        #### For double panel plot
		dvar = np.median(contri)        
		dout = leastsq(res, dvar, args=(xfinal,contri,100))
		dyfit= const(xfinal, dout[0][0])       

        
		redchisq = chisq/(len(xfinal)-2)
		ax1= fig.add_subplot(2,1,1)
		ax2= fig.add_subplot(4,1,3)

        
		if redchisq > redchilimit:
			ax1.plot(xfinal+starttime,yfit,label="Best Fit Model Redchisq="+str(redchisq))
			ax1.errorbar(xfinal+starttime,yfinal,yerr=np.sqrt(yfinal),fmt=".",label="Data")
			fig.suptitle("Stare "+str(count+1))
			ax1.legend()
			ax1.set_ylabel("Total counts (in each bin)")
			ax1.set_xlabel('MJD')                                    
			ax2.plot(xfinal+starttime, y0, color='red')
			ax2.errorbar(xfinal+starttime, contri,yerr=np.sqrt(y1), label='chisq/data point',fmt='.', ls= 'None', color='k' )  
			ax2.set_ylabel('$(model-data)^2/err^2$')            
			ax2.set_xlabel('MJD')                        
			fig.savefig("1_Fred_Fit_"+str(count+1)+".png")
			plt.close(fig)
			detections.append((xfinal[0]+starttime,xfinal[len(xfinal)-1]+starttime))
			confidence.append(redchisq)
	plt.close()            
	count += 1
	prog = (float(count)/float(len(stares)))*100
	sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
	print('\n')    
	sys.stdout.flush()
sys.stdout.write("\n")

np.savetxt("1_confidence_"+sizesname[-7:], confidence)

### Fitting a constant line to remove the unnecessary files which have constant data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys

##### Sorting lists of (list1= max(JD), list2 = min(JD)) for different FITS files from the file stares created above
##### The sortlist() arranges the list2 in order s.t. the value corresponding to the list1 is in increasing order 

def sort_list(list1, list2): 
    zipped_pairs = zip(list2, list1) 
    z = [x for _, x in sorted(zipped_pairs)]       
    return z 

def const(t, c):
    return  t*0+ c #+minvalue

def res(var, t, data,eps_data):
    c= var[0]
    model = t*0 + c #+minvalue
    return (data-model) / eps_data

def residual(variables, t, data, eps_data):
    tau1 = variables[0]
    tau2 = variables[1]
    t_s  = 0 #variables[2]
    model = minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))
    return (data-model) / eps_data

####
#### This is the Norris function which is required for the fitting purpose 
#### We are trying to set the start of the function 

def func(t,tau1,tau2,t_s=0):
	return minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))

#def exp(k,ydata,c):
#    k= variables[0]
#    c= variables[1]
#    Y= np.max(ydata)*np.exp(-(k)) + c

filename = 'Stare_Orbit_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")         ##############
sizesname = 'Stare_Orbit_stares_5s.txt'#input("Enter path to file containing stare intervals: ")            ##############
redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

count = 0

numbers=[]
#print(len(stares))
while count < len(stares):
	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[count].split()[0])      #### start time of the stares for 1 event min(JD)
	end = float(stares[count].split()[1])        #### end time of the stares for 1 event max(JD)
	Orbit = float(stares[count].split()[2])      #### orbit number 
	Stare = float(stares[count].split()[3])      #### Stare Starting    
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:            #### If the lines[k] (MJD values) is between the starting and the ending point
			xdata.append(float(p[0]))            #### the MJD value and its corresponding counts is appended
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)           #### sorting ydata s.t. xdata is in the inc order.
		xdata = sorted(xdata)                    #### xdata is in inc order, and corrrespoindingly ydata is provided
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime        #### for fitting purpose, the xdata[0] = 0 for the FRED fit
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)
		var = np.mean(ydata)
		sigma= np.std(ydata) 
#		print('stddev = {}'.format(sigma), 'mean= {}'.format(var))         
		value= var-2*sigma
#		print(count+1, '\t','2$\sigma$ level min =',value)        
		indices=np.where(ydata>value)
#		print(indices)    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
#		print(len(xfinal), len(xdata))        
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)     
#		print(len(yfinal), len(ydata))        
		out = leastsq(res, var, args=(xfinal,yfinal,100))        
		yfit = const(xfinal, out[0][0])#,out[0][1])#,out[0][2])  
#		print(out)        
        
#		variables = [0.01, 0.00015]#,0]
#		out = leastsq(residual, variables, args=(xdata, ydata, 1000))
#		yfit = func(xdata,out[0][0],out[0][1])#,out[0][2])
		y1 = yfinal
#		print('\n')        
#		print(ydata,'\n',xdata+starttime,'\n','length of data',len(ydata),',',len(xdata))        
		y2 = yfit
		ychi = []   

#		print(y1,y2)                
        
		for i in range(len(xfinal)):
			if y1[i] != 0:
				ychi.append((y1[i]-y2[i])**2/(y1[i]))
		chisq = sum(ychi)
		contri= (y1-y2)**2/(y1)
#		print('error= ', np.sqrt(yfinal)[2])        
#		for j in range(len(xdata)):
#			if y1[j] ==0:
#				contri.append(float(0)) 
#			else:
#				contri.append(chisq/ydata)
		y0=np.zeros(shape= np.shape(contri)) 

		error= 2*(abs(y1-y2))/np.sqrt(y1)
            
        #### For double panel plot
		dvar = np.median(contri)        
		dout = leastsq(res, dvar, args=(xfinal,contri,100))
		dyfit= const(xfinal, dout[0][0])       

        
		redchisq = chisq/(len(xfinal)-1) #2)   ### rechisq= chisq/dof ; dof= tot(points) - parameter
		ax1= fig.add_subplot(2,1,1)
		ax2= fig.add_subplot(4,1,3)

#		print(xfinal,yfit)
        
		if redchisq > redchilimit:
			ax1.plot(xfinal+starttime,yfit,label="Best Fit Model Redchisq="+str(redchisq))
			ax1.errorbar(xfinal+starttime,yfinal,yerr=np.sqrt(yfinal),fmt=".",label="Data",color='orange')
			fig.suptitle("Orbit "+ str(Orbit)+"     Stare "+str(count+Stare) )    ##################
			ax1.legend()
			ax1.set_ylabel("Total counts (in each bin)")
			ax1.set_xlabel('MJD')                                    
			ax2.plot(xfinal+starttime, y0, color='red')
			ax2.errorbar(xfinal+starttime, contri,yerr=error, label='chisq/data point',fmt='.', ls= 'None', color='k' )  
			ax2.set_ylabel('$(model-data)^2/data$')            
			ax2.set_xlabel('MJD')                        
			fig.savefig("Orbit_"+str(Orbit)+"_Stare_"+ str(count+Stare)+"_Const_Fit_"+str(count+1)+".png")               ######################
			plt.close(fig)
			detections.append((xfinal[0]+starttime,xfinal[len(xfinal)-1]+starttime))
			confidence.append(redchisq)
			numbers.append(count)            
	plt.close()            
	count += 1
	prog = (float(count)/float(len(stares)))*100
	sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
	print('\n')    
	sys.stdout.flush()
sys.stdout.write("\n")
print(numbers)
np.savetxt("Orbit_"+str(Orbit)+"_Stare_"+ str(Stare)+"_confidence_"+sizesname[-7:], confidence)              ######################

### Exponential Fit $y = ae^{-bx} + c $ to selected files 

In [ ]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys


def exp(xdata,b):
#    A= var[0]
#    b= var[1]
#    c= var[1]
    return minvalue+ (maxvalue-minvalue)*np.exp(-b*(xdata))

def resexp(var, x, data, eps_data):
    b = var[0]
#    t_s  = 0 #variables[2]
    model = minvalue+ (maxvalue-minvalue)*np.exp(-b*(x))
    return (data-model) / eps_data

def resexp(var, x, data, eps_data):
    b = var[0]
#    t_s  = 0 #variables[2]
    model = minvalue+ (maxvalue-minvalue)*np.exp(-b*(x))
    return (data-model) / eps_data


filename = 'Orbit_'+str(Orbit_number)+'_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")
sizesname = 'Orbit_'+str(Orbit_number)+'_stares_5s.txt'#input("Enter path to file containing stare intervals: ")
redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

count = 0


for n in numbers:
	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[n].split()[0])      #### start time of the stares for 1 event min(JD)
	end = float(stares[n].split()[1])        #### end time of the stares for 1 event max(JD)
	Orbit = float(stares[count].split()[2])      #### orbit number          
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:            #### If the lines[k] (MJD values) is between the starting and the ending point
			xdata.append(float(p[0]))            #### the MJD value and its corresponding counts is appended
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)           #### sorting ydata s.t. xdata is in the inc order.
		xdata = sorted(xdata)                    #### xdata is in inc order, and corrrespoindingly ydata is provided
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime        #### for fitting purpose, the xdata[0] = 0 for the FRED fit
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)        
		mean=np.mean(ydata)        
#		var = np.mean(ydata)
#		c= np.max(ydata)        
		sigma= np.std(ydata) 
##		print('stddev = {}'.format(sigma), 'mean= {}'.format(var))         
		value= mean-2*sigma
##		print(n+1, '\t','2$\sigma$ level min =',value)        
		indices=np.where(ydata>value)
#		print(indices)    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
#		print(len(xfinal), len(xdata))        
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)     

		var=[0.0001]     

###		popt,pcov=curve_fit(exp,starttime+(xfinal),yfinal,(var[0]))#,var[2]))        
		out = leastsq(resexp, 0.000001, args=(xfinal,yfinal,1000)) 
        
		print(out)        
		yfit = exp(xfinal,out[0][0])# ,out[0][1])#,out[0][1])#,out[0][2])  
#		print(out)                
#		variables = [0.01, 0.00015]#,0]
#		out = leastsq(residual, variables, args=(xdata, ydata, 1000))
#		yfit = func(xdata,out[0][0],out[0][1])#,out[0][2])

		y1 = yfinal
		y2 = yfit
		ychi = []   
               
        
		for i in range(len(xfinal)):
			if y1[i] != 0:
				ychi.append((y1[i]-y2)**2/(y1[i]))
		chisq = sum(ychi)
		contri= (y1-y2)**2/(y1)
#		print('error= ', np.sqrt(yfinal)[2])        
#		for j in range(len(xdata)):
#			if y1[j] ==0:
#				contri.append(float(0)) 
#			else:
#				contri.append(chisq/ydata)
		y0=np.zeros(shape= np.shape(contri)) 

		error= 2*(abs(y1-y2))/np.sqrt(y1)

        
#		print(yfinal,yfit)

        #### For double panel plot
		dvar = np.median(contri)      
		dout = leastsq(res, dvar, args=(xfinal,contri,100))
		dyfit= exp(xfinal, dout[0][0])# dout[0][1])       

        
		redchisq = chisq/(len(xfinal)-1) #2)   ### rechisq= chisq/dof ; dof= tot(points) - parameter
		ax1= fig.add_subplot(2,1,1)
		ax2= fig.add_subplot(4,1,3)

        
#		if redchisq > redchilimit:
		ax1.plot(xfinal+starttime,yfit,label="Best Fit Model Redchisq="+str(redchisq))
		ax1.errorbar(xfinal+starttime,yfinal,yerr=np.sqrt(yfinal),fmt=".",label="Data",color='orange')
			fig.suptitle("Orbit "+ str(Orbit)+"     Stare "+str(count+2) )    ##################
##		ax1.legend()
		ax1.set_ylabel("Total counts (in each bin)")
		ax1.set_xlabel('MJD')                                    
		ax2.plot(xfinal+starttime, y0, color='red')
		ax2.errorbar(xfinal+starttime, contri,yerr=error, label='chisq/data point',fmt='.', ls= 'None', color='k' )  
		ax2.set_ylim(-5,30)        
		ax2.set_ylabel('$(model-data)^2/data$')            
		ax2.set_xlabel('MJD')                        
		fig.savefig("Orbit_"+str(Orbit)+"_exp_Fit_"+str(n+1)+".png")           ###################
		plt.close(fig)
		detections.append((xfinal[0]+starttime,xfinal[len(xfinal)-1]+starttime))
		confidence.append(redchisq)
#			numbers.append(n+1)            
	plt.close()            
	n += 1
	prog = (float(n)/float(len(stares)))*100
	sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
	print('\n')    
	sys.stdout.flush()
sys.stdout.write("\n")
print(numbers)
np.savetxt("Orbit_"+str(Orbit)+"_expconfidence_"+sizesname[-7:], confidence,fmt='%s')     #################

### Implementing Trigger Algorithm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys


##### Sorting lists of (list1= max(JD), list2 = min(JD)) for different FITS files from the file stares created above
##### The sortlist() arranges the list2 in order s.t. the value corresponding to the list1 is in increasing order 


def sort_list(list1, list2):                    
	zipped_pairs = zip(list2, list1) 
	z = [x for _, x in sorted(zipped_pairs)]       
	return z 

def const(t, c):
	return  t*0+ c #+minvalue

def res(var, t, data,eps_data):
	c= var[0]
	model = t*0 + c #+minvalue
	return (data-model) / eps_data

def residual(variables, t, data, eps_data):
	tau1 = variables[0]
	tau2 = variables[1]
	t_s  = variables[2]
	model = minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))
	return (data-model) / eps_data

def func(t,tau1,tau2,t_s):
	return minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t-t_s))-((t-t_s)/tau2))


def residual1(variables, t, data, eps_data):
	tau1 = variables[0]
	tau2 = variables[1]
#	t_s  = 0 #variables[2]
	model = minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t))-((t)/tau2))
	return (data-model) / eps_data

def func1(t,tau1,tau2):
	return minvalue + (maxvalue-minvalue)*(np.exp(2*np.sqrt(tau1/tau2)))*np.exp((-tau1/(t))-((t)/tau2))


filename = '1_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")
sizesname = '1_stares_5s.txt'#input("Enter path to file containing stare intervals: ")
#redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

for n in numbers:
#	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[n].split()[0])
	end = float(stares[n].split()[1])
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:
			xdata.append(float(p[0]))
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)
		xdata = sorted(xdata)
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)
		var = np.median(ydata)
		sigma= np.std(ydata) 
		print('stddev = {}'.format(sigma), 'mean= {}'.format(var))         
		value= var-2*sigma
		print(n, '\t',value)        
		indices=np.where(ydata>value)
#		print(indices)    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)    
######################################################################
######################################################################
		t_s=0
		for k in range(1,len(yfinal)-4):
			if ((yfinal[k+4]-(yfinal[k]+yfinal[k+1]+yfinal[k+2]+yfinal[k+3])/4)>2*sigma):
				t_s =xfinal[k]+starttime
			else:
				pass
	print('t_s =',t_s)
	print('-'*100)    



### Linear fits $y = mx + c$   

In [ ]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys

def line(m,x):
    return np.min(yfinal)+ m*x
    
def resline(var, x, data, eps_data):
    m = var[0]
    model = np.min(yfinal)+ m*x
    return (data-model) / eps_data


filename = '2_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")
sizesname = '2_stares_5s.txt'#input("Enter path to file containing stare intervals: ")
redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

count = 0


num=[304,323,484]
for n in num:
	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[n].split()[0])      #### start time of the stares for 1 event min(JD)
	end = float(stares[n].split()[1])        #### end time of the stares for 1 event max(JD)
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:            #### If the lines[k] (MJD values) is between the starting and the ending point
			xdata.append(float(p[0]))            #### the MJD value and its corresponding counts is appended
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)           #### sorting ydata s.t. xdata is in the inc order.
		xdata = sorted(xdata)                    #### xdata is in inc order, and corrrespoindingly ydata is provided
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime        #### for fitting purpose, the xdata[0] = 0 for the FRED fit
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)        
		mean=np.mean(ydata)        
        
		sigma= np.std(ydata) 
        
		value= mean-2*sigma
##		print(n+1, '\t','2$\sigma$ level min =',value)        
		indices=np.where(ydata>value)
#		print(indices)    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
#		print(len(xfinal), len(xdata))        
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)     

		var=[1]     

###		popt,pcov=curve_fit(exp,starttime+(xfinal),yfinal,(var[0]))#,var[2]))        
		out = leastsq(resline, var, args=(xfinal,yfinal,1000)) 
        
		print(out)        
		yfit = line(xfinal,out[0][0]) #,out[0][1])#,out[0][2])  
#		print(out)                
#		variables = [0.01, 0.00015]#,0]
#		out = leastsq(residual, variables, args=(xdata, ydata, 1000))
#		yfit = func(xdata,out[0][0],out[0][1])#,out[0][2])

		y1 = yfinal
		y2 = yfit
		ychi = []   
               
        
		for i in range(len(xfinal)):
			if y1[i] != 0:
				ychi.append((y1[i]-y2)**2/(y1[i]))
		chisq = sum(ychi)
		contri= (y1-y2)**2/(y1)
#		print('error= ', np.sqrt(yfinal)[2])        
#		for j in range(len(xdata)):
#			if y1[j] ==0:
#				contri.append(float(0)) 
#			else:
#				contri.append(chisq/ydata)
		y0=np.zeros(shape= np.shape(contri)) 

		error= 2*(abs(y1-y2))/np.sqrt(y1)

        
#		print(yfinal,yfit)

        #### For double panel plot
		dvar = np.median(contri)      
		dout = leastsq(resline, dvar, args=(xfinal,contri,100))
		dyfit= line(xfinal, dout[0][0]) #dout[0][1])       

        
		redchisq = chisq/(len(xfinal)-1) #2)   ### rechisq= chisq/dof ; dof= tot(points) - parameter
		ax1= fig.add_subplot(2,1,1)
		ax2= fig.add_subplot(4,1,3)

        
#		if redchisq > redchilimit:
		ax1.plot(xfinal+starttime,yfit,label="Best Fit Model Redchisq="+str(redchisq))
		ax1.errorbar(xfinal+starttime,yfinal,yerr=np.sqrt(yfinal),fmt=".",label="Data",color='orange')
		fig.suptitle("Stare "+str(n+1))
##		ax1.legend()
		ax1.set_ylabel("Total counts (in each bin)")
		ax1.set_xlabel('MJD')                                    
		ax2.plot(xfinal+starttime, y0, color='red')
		ax2.errorbar(xfinal+starttime, contri,yerr=error, label='chisq/data point',fmt='.', ls= 'None', color='k' )  
		ax2.set_ylim(-5,30)        
		ax2.set_ylabel('$(model-data)^2/data$')            
		ax2.set_xlabel('MJD')                        
		fig.savefig("line2_Fred_Fit_"+str(n+1)+".png")
		plt.close(fig)
		detections.append((xfinal[0]+starttime,xfinal[len(xfinal)-1]+starttime))
		confidence.append(redchisq)
#			numbers.append(n+1)            
	plt.close()            
	n += 1
	prog = (float(n)/float(len(stares)))*100
	sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
	print('\n')    
	sys.stdout.flush()
sys.stdout.write("\n")
#print(numbers)
#np.savetxt("1_expconfidence_"+sizesname[-7:], confidence,fmt='%s')

### Step Function fits to some files  

In [1]:
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import leastsq
import sys

def f(p,y): 
    return p +p*(np.sign(y-p)) 
    
def resf(var, data, eps_data):
    p = var[0]
    model = p+ p*(np.sign(data-p))
    return (data-model) / eps_data


filename = '1_lcurve_binned_stare_5s.txt'#input("Enter path to binned lightcurve file: ")
sizesname = '1_stares_5s.txt'#input("Enter path to file containing stare intervals: ")
redchilimit = 2 #float(input("Enter limit of redchisq above which to reject fits: "))

fp = open(filename, "r")
lines = fp.readlines()
fs = open(sizesname, "r")
stares = fs.readlines()
detections = []
confidence = []

count = 0


num=[467]
for n in num:
	fig = plt.figure(figsize=[8,12])
	xdata = []
	ydata = []
	start = float(stares[n].split()[0])      #### start time of the stares for 1 event min(JD)
	end = float(stares[n].split()[1])        #### end time of the stares for 1 event max(JD)
	k = 1
	for k in range(len(lines)):
		p = lines[k].split()
		if start < float(p[0]) < end:            #### If the lines[k] (MJD values) is between the starting and the ending point
			xdata.append(float(p[0]))            #### the MJD value and its corresponding counts is appended
			ydata.append(float(p[1]))
	if len(xdata)>0:
		ydata = sort_list(ydata,xdata)           #### sorting ydata s.t. xdata is in the inc order.
		xdata = sorted(xdata)                    #### xdata is in inc order, and corrrespoindingly ydata is provided
		starttime = xdata[0]
		xdata = np.array(xdata)-starttime        #### for fitting purpose, the xdata[0] = 0 for the FRED fit
		ydata = np.array(ydata)
		meanvalue = np.mean(ydata)
		maxvalue = max(ydata)
		minvalue = min(ydata)        
		mean=np.mean(ydata)        
        
		sigma= np.std(ydata) 
        
		value= mean-2*sigma
      
		indices=np.where(ydata>value)
    
		xf=[]
		yf=[]
		for i in indices:
			xf.append(xdata[i])
			yf.append(ydata[i])
		samplex = np.array(xf)
		xfin = samplex.flatten()
		xfinal = np.delete(xfin,len(np.array(xfin))-1,None)             
        
		sampley = np.array(yf)
		yfin = sampley.flatten()
		yfinal = np.delete(yfin,len(np.array(yfin))-1,None)     
   

		p= (np.max(yfinal)-np.min(yfinal)) 
		print(p)        
		popt,pcov=curve_fit(f,p,yfinal)

		y1 = yfinal
		y2 = yfit
		ychi = []   
               
        
#		for i in range(len(xfinal)):
#			if y1[i] != 0:
#				ychi.append((y1[i]-y2)**2/(y1[i]))
#		chisq = sum(ychi)
#		contri= (y1-y2)**2/(y1)
#		print('error= ', np.sqrt(yfinal)[2])        
#		for j in range(len(xdata)):
#			if y1[j] ==0:
#				contri.append(float(0)) 
#			else:
#				contri.append(chisq/ydata)
		y0=np.zeros(shape= np.shape(contri)) 

#		error= 2*(abs(y1-y2))/np.sqrt(y1)

        
#		print(yfinal,yfit)

        #### For double panel plot
#		dvar = np.median(contri)      
#		dout = leastsq(resline, dvar, args=(xfinal,contri,100))
#		dyfit= line(xfinal, dout[0][0]) #dout[0][1])       

        
		redchisq = chisq/(len(xfinal)-1) #2)   ### rechisq= chisq/dof ; dof= tot(points) - parameter
		ax1= fig.add_subplot(2,1,1)
#		ax2= fig.add_subplot(4,1,3)

        
#		if redchisq > redchilimit:
		ax1.plot(xfinal+starttime,f(*popt, yfinal),label="Best Fit Model Redchisq="+str(redchisq))
		ax1.errorbar(xfinal+starttime,yfinal,yerr=np.sqrt(yfinal),fmt=".",label="Data",color='orange')
		fig.suptitle("Stare "+str(n+1))
##		ax1.legend()
		ax1.set_ylabel("Total counts (in each bin)")
		ax1.set_xlabel('MJD')
#		ax1.set_ylim(-200,200)
#		ax2.plot(xfinal+starttime, y0, color='red')
#		ax2.errorbar(xfinal+starttime, contri,yerr=error, label='chisq/data point',fmt='.', ls= 'None', color='k' )  
#		ax2.set_ylim(-5,30)        
#		ax2.set_ylabel('$(model-data)^2/data$')            
#		ax2.set_xlabel('MJD')                        
#		fig.savefig("Step1_Fred_Fit_"+str(n+1)+".png")
#		plt.close(fig)
		detections.append((xfinal[0]+starttime,xfinal[len(xfinal)-1]+starttime))
		confidence.append(redchisq)
#			numbers.append(n+1)            
#	plt.close()            
	n += 1
	prog = (float(n)/float(len(stares)))*100
	sys.stdout.write("\r" + "#"*int(prog/2) + repr(prog)[0:4] + "%")
	print('\n')    
	sys.stdout.flush()
sys.stdout.write("\n")
#print(numbers)
#np.savetxt("1_expconfidence_"+sizesname[-7:], confidence,fmt='%s')

FileNotFoundError: [Errno 2] No such file or directory: '1_lcurve_binned_stare_5s.txt'